#Task 2- Transfer Learning on CIFAR10

##Name-Swastik Mahapatra

Here i have Implemented Transfer Learning On the Cifar10 dataset using Resnet50. And i have showcased the retraining of the Fully Connected Layer to do Transfer Learning After passing the images through convolutional layers of Resnet50, i have limited the number of training samples to 7000 and the number of testing samples to 1000. and have them showcased the results from the training process.

Importing Required Libraries

In [1]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
import os
import numpy as np
import cv2

Using TensorFlow backend.


##Extracting and pre-processing Data to train

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


##Importing the Resnet Architecture

In [0]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_acc', 
                                           min_delta=0, 
                                           patience=5, 
                                           verbose=0, 
                                           mode='auto')
resnet_base = keras.applications.resnet50.ResNet50(include_top=False, 
                                     weights='imagenet', pooling = 'avg')

In [4]:
resnet_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

##Taking 5000 Examples and reshaping it to Resnet capable Size

Have set number of training examples to 5000 because of lack of computing capacity

In [0]:
#Reshaping Training Set to meet Resnet50 Requirements
nb_train_samples=7000
img_rows=255
img_cols=255
if K.image_dim_ordering() == 'th':
  big_x_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in x_train[:nb_train_samples,:,:,:]])
else:
  big_x_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in x_train[:nb_train_samples,:,:,:]])

In [4]:
big_x_train.shape

(7000, 255, 255, 3)

Predicting Train Values on Resnet50

In [0]:
x_preds_res_train = resnet_base.predict(big_x_train)

In [7]:
print(str(x_preds_res_train))

[[0.         0.         0.01127299 ... 1.5338063  0.         0.        ]
 [0.         0.         0.01208845 ... 1.6604104  0.         0.        ]
 [0.         0.         0.01489897 ... 1.5456302  0.         0.        ]
 ...
 [0.         0.         0.01113216 ... 1.6085738  0.         0.        ]
 [0.         0.         0.02486641 ... 1.6127249  0.         0.        ]
 [0.         0.         0.01168566 ... 1.5668232  0.         0.        ]]


The number of test samples has been brought down to 500 unit because of lack of compute capacity

In [0]:
#To reshape the Test samples as per Resnet50 Requirements
nb_train_samples=1000
img_rows=255
img_cols=255
if K.image_dim_ordering() == 'th':
  big_x_test = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in x_test[:nb_train_samples,:,:,:]])
else:
  big_x_test = np.array([cv2.resize(img, (img_rows,img_cols)) for img in x_test[:nb_train_samples,:,:,:]])

Predicting Test values on Resnet50

In [0]:
x_preds_res_test = resnet_base.predict(big_x_test)

In [10]:
print(x_preds_res_test)

[[0.         0.         0.01085033 ... 1.6115465  0.         0.        ]
 [0.         0.         0.01187322 ... 1.6169493  0.         0.        ]
 [0.         0.         0.01299578 ... 1.567474   0.         0.        ]
 ...
 [0.         0.         0.01278098 ... 1.7618841  0.         0.        ]
 [0.         0.         0.01098434 ... 1.6152694  0.         0.        ]
 [0.         0.         0.01254137 ... 1.5315981  0.         0.        ]]


In [0]:
#Resizing/flattening the arrays to be fed to fully connected layer
import pickle
pickle.dump(x_preds_res_train, open("x_preds_res_train", "wb"))
pickle.dump(x_preds_res_test, open("x_preds_res_test", "wb"))

In [12]:
x_preds_res_train.shape[1:]

(2048,)

Defining the fully connected layer for Transfer Learning

In [0]:
#Finetuning the Resnet Model
output =  Sequential()
output.add(Dense(512, input_shape=x_preds_res_train.shape[1:]))
output.add(Activation('relu'))
output.add(Dense(10))
output.add(Activation('softmax'))

Implementing Transfer Learning on Cifar10 after passing through Resnet50

In [15]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-5)

output.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

history_resnet = output.fit(x_preds_res_train, y_train[:7000],
              batch_size=64,
              epochs=600,
              validation_data=(x_preds_res_test, y_test[:1000]),
              shuffle=True,
              callbacks=[early_stop])

Train on 7000 samples, validate on 1000 samples
Epoch 1/600
7000/7000 [==============================] - 2s 273us/step - loss: 2.3200 - acc: 0.1040 - val_loss: 2.3166 - val_acc: 0.1030
Epoch 2/600
7000/7000 [==============================] - 1s 114us/step - loss: 2.3103 - acc: 0.1110 - val_loss: 2.3067 - val_acc: 0.1090
Epoch 3/600
7000/7000 [==============================] - 1s 122us/step - loss: 2.3071 - acc: 0.1076 - val_loss: 2.3101 - val_acc: 0.1120
Epoch 4/600
7000/7000 [==============================] - 1s 125us/step - loss: 2.3001 - acc: 0.1126 - val_loss: 2.3219 - val_acc: 0.0860
Epoch 5/600
7000/7000 [==============================] - 1s 122us/step - loss: 2.2973 - acc: 0.1150 - val_loss: 2.3128 - val_acc: 0.1060
Epoch 6/600
7000/7000 [==============================] - 1s 122us/step - loss: 2.2921 - acc: 0.1230 - val_loss: 2.3023 - val_acc: 0.1190
Epoch 7/600
7000/7000 [==============================] - 1s 121us/step - loss: 2.2877 - acc: 0.1313 - val_loss: 2.2769 - val_acc: 

7000/7000 [==============================] - 1s 123us/step - loss: 2.2816 - acc: 0.1271 - val_loss: 2.2884 - val_acc: 0.1000
Epoch 9/600
7000/7000 [==============================] - 1s 121us/step - loss: 2.2788 - acc: 0.1423 - val_loss: 2.2839 - val_acc: 0.1120
Epoch 10/600
7000/7000 [==============================] - 1s 120us/step - loss: 2.2727 - acc: 0.1414 - val_loss: 2.2657 - val_acc: 0.1390
Epoch 11/600
7000/7000 [==============================] - 1s 121us/step - loss: 2.2679 - acc: 0.1497 - val_loss: 2.2656 - val_acc: 0.1530
Epoch 12/600
7000/7000 [==============================] - 1s 122us/step - loss: 2.2630 - acc: 0.1574 - val_loss: 2.2683 - val_acc: 0.0930


The Above Results Showcase How quick increase in accuracy can be obtained by training the CIFAR10 Images using Transfer Learning

#........END........